In [112]:

# bundle = {"application/vnd.vega.v5+json": schema}
# print("", end=None)
# IPython.display.display(bundle, raw=True)





In [2]:
import networkx as nx
import numpy as np
import pandas as pd
from pyciemss.integration_utils.result_processing import convert_to_output_format
from pyciemss.visuals import  vega, plots
from pathlib import Path
import os
import json 

from IPython.display import display

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/threadpoolctl.py:1223: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [3]:
import json
import random
from itertools import chain
from pathlib import Path

import networkx as nx
import numpy as np
import pandas as pd
import pytest

import pyciemss
from pyciemss.integration_utils.result_processing import convert_to_output_format
from pyciemss.visuals import plots, vega



In [4]:
def distributions():
    model_1_path = (
            "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration"
            "/main/data/models/SEIRHD_NPI_Type1_petrinet.json"
    )
    start_time = 0.0
    end_time = 100.0
    logging_step_size = 1
    num_samples = 30
    sample = pyciemss.sample(
        model_1_path,
        end_time,
        logging_step_size,
        num_samples,
        start_time=start_time,
        solver_method="euler",
    )["unprocessed_result"]

    for e in sample.values():
        if len(e.shape) > 1:
            num_timepoints = e.shape[1]

    return convert_to_output_format(
        sample,
        timepoints=np.linspace(start_time, end_time, num_timepoints),
        time_unit="notional",
    )

distributions = distributions()
print(distributions.head())
print(distributions.columns)
all_columns = ["timepoint_id", "sample_id", 'infected_observable_state',
       'exposed_observable_state', 'hospitalized_observable_state',
       'dead_observable_state']
all_columns = ["timepoint_id", "sample_id", 'infected_observable_state']

distributions = distributions[all_columns]

   timepoint_id  sample_id  persistent_beta_c_param  persistent_kappa_param  \
0             0          0                 0.792868                0.355971   
1             1          0                 0.792868                0.355971   
2             2          0                 0.792868                0.355971   
3             3          0                 0.792868                0.355971   
4             4          0                 0.792868                0.355971   

   persistent_gamma_param  persistent_hosp_param  persistent_death_hosp_param  \
0                0.116676               0.161222                     0.051188   
1                0.116676               0.161222                     0.051188   
2                0.116676               0.161222                     0.051188   
3                0.116676               0.161222                     0.051188   
4                0.116676               0.161222                     0.051188   

   persistent_I0_param   D_state    E_

In [116]:
# def traces(distributions):
#     return (
#         distributions[distributions["sample_id"] == 0]
#         .set_index("timepoint_notional")[["dead_observable_state", "I_state"]]
#         .rename(
#             columns={
#                 "dead_observable_state": "dead_exemplar",
#                 "I_state": "I_exemplar",
#             }
#         )
#     )
# traces = traces(distributions)
# traces

In [5]:
%matplotlib inline

_output_root = "output_images/"

def save_result(data, name, ref_ext):
    """Save new reference files"""
    # _output_root.mkdir(parents=True, exist_ok=True)

    mode = "w" if ref_ext == "svg" else "wb"
    with open(os.path.join(_output_root, f"{name}.{ref_ext}"), mode) as f:
        f.write(data)
traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(distributions, select_by = select_by,  kmean=False)
    traces_list.append(examplary_line)

new_df = pd.concat(traces_list, axis = 1)


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=57.9980 , p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=759.0921, p=0.0000  , df=10
likelihood ratio test: chi2=200.6374, p=0.0000  , df=10
parameter F test:         F=57.8450 , p=0.0000  , df_denom=68, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=12.5078 , p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=163.7056, p=0.0000  , df=10
likelihood ratio test: chi2=92.8794 , p=0.0000  , df=10
parameter F test:         F=12.5113 , p=0.0000  , df_denom=68, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=5.3391  , p=0.0000  , df_denom=68, df_num=10
ssr based chi2 test:   chi2=69.8789 , p=0.0000  , df=10
likelihood ratio test: chi2=51.5761 , p=0.0000  , df=10
parameter F test:         F=5.6117  , p=0.0000  , df_denom=68, df_num=9

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=9.3416  , p

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 9
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

In [6]:
mean_line_df 

trajectory,infected_observable_state_compare_line_no_cluster
timepoint,
0,15.404147
1,19.096250
2,21.363398
3,23.200246
4,25.035779
...,...
94,357288.143164
95,344345.879027
96,332529.691912


In [7]:
combined_df = pd.concat([new_df, mean_line_df], axis =1)
combined_df

trajectory,infected_observable_state_mean_no_cluster,infected_observable_state_var_no_cluster,infected_observable_state_granger_no_cluster,infected_observable_state_compare_line_no_cluster
timepoint,,,,
0,18.839336,18.839336,2.180719e+01,15.404147
1,24.788004,24.788004,2.778604e+01,19.096250
2,29.499735,29.499735,3.301510e+01,21.363398
3,33.763748,33.763748,3.821029e+01,23.200246
4,38.003677,38.003677,4.375787e+01,25.035779
...,...,...,...,...
94,321817.250000,321817.250000,2.377996e+06,357288.143164
95,324220.718750,324220.718750,2.474322e+06,344345.879027
96,325817.937500,325817.937500,2.564101e+06,332529.691912


In [8]:
schema = plots.trajectories(distributions, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "traj_infected_observable", "png")


FileNotFoundError: [Errno 2] No such file or directory: 'output_images/traj_infected_observable.png'

In [10]:
schema = plots.trajectories(distributions, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "traj_infected_observable", "png")


FileNotFoundError: [Errno 2] No such file or directory: 'output_images/traj_infected_observable.png'

In [9]:
#'timepoint_id', 'sample_id'
sin_distribution = pd.read_csv("paper_sin_05/paper_sin_05_new_df.csv")
sin_distribution = sin_distribution.rename(
            columns={
                'Unnamed: 0.1': "timepoint_id",
                'Unnamed: 0': "100",
            }
        )
sin_distribution = sin_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")

traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = select_by,  kmean=False)
    traces_list.append(examplary_line)


new_df = pd.concat(traces_list, axis = 1)
combined_df = pd.concat([new_df, mean_line_df], axis =1)

schema = plots.trajectories(sin_distribution, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.5077  , p=0.1554  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=19.6658 , p=0.0326  , df=10
likelihood ratio test: chi2=17.7865 , p=0.0587  , df=10
parameter F test:         F=1.5077  , p=0.1554  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.7838  , p=0.0800  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=23.2664 , p=0.0098  , df=10
likelihood ratio test: chi2=20.6939 , p=0.0233  , df=10
parameter F test:         F=1.7838  , p=0.0800  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=0.5552  , p=0.8443  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=7.2420  , p=0.7024  , df=10
likelihood ratio test: chi2=6.9653  , p=0.7287  , df=10
parameter F test:         F=0.5552  , p=0.8443  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.1772  , 

In [11]:
#'timepoint_id', 'sample_id'
sin_distribution = pd.read_csv("paper_sin_05/paper_sin_05_new_df.csv")
sin_distribution = sin_distribution.rename(
            columns={
                'Unnamed: 0.1': "timepoint_id",
                'Unnamed: 0': "100",
            }
        )
sin_distribution = sin_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")

traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = select_by,  kmean=True)
    traces_list.append(examplary_line)


new_df = pd.concat(traces_list, axis = 1)
combined_df = pd.concat([new_df, mean_line_df], axis =1)

schema = plots.trajectories(sin_distribution, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")

Init 1


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    1.3s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s


11.712 --> 

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s


6.480 --> 

...

5.477 --> 

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s


5.451 --> 5.451 --> 

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.0809  , p=0.3888  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=14.0981 , p=0.1686  , df=10
likelihood ratio test: chi2=13.0972 , p=0.2183  , df=10
parameter F test:         F=1.0809  , p=0.3888  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=4.0779  , p=0.0002  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=53.1903 , p=0.0000  , df=10
likelihood ratio test: chi2=41.7928 , p=0.0000  , df=10
parameter F test:         F=4.0779  , p=0.0002  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.0999  , p=0.3748  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=14.3462 , p=0.1578  , df=10
likelihood ratio test: chi2=13.3114 , p=0.2068  , df=10
parameter F test:         F=1.0999  , p=0.3748  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s


ssr based F test:         F=0.6543  , p=0.7621  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=8.5343  , p=0.5768  , df=10
likelihood ratio test: chi2=8.1535  , p=0.6138  , df=10
parameter F test:         F=0.6543  , p=0.7621  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=2.1951  , p=0.0281  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=28.6319 , p=0.0014  , df=10
likelihood ratio test: chi2=24.8594 , p=0.0056  , df=10
parameter F test:         F=2.1951  , p=0.0281  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.7091  , p=0.0961  , df_denom=69, df_num=10
ssr based chi2 test:   chi2=22.2929 , p=0.0137  , df=10
likelihood ratio test: chi2=19.9171 , p=0.0300  , df=10
parameter F test:         F=1.7091  , p=0.0961  , df_denom=69, df_num=10

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=1.2816  , p=0.2581  , df_denom=69, df_num=10
ssr based ch

In [10]:
#'timepoint_id', 'sample_id'
sin_distribution = pd.read_csv("paper_sin/paper_sin_new_df.csv")
sin_distribution = sin_distribution.rename(
            columns={
                'Unnamed: 0': "timepoint_id",
            }
        )
sin_distribution = sin_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")


traces = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "mean")[0]
traces = traces.rename(
            columns={
                'same_stat_state': "traces_mean",
            }
        )
# traces['traces_var'] = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "var")[0]["same_stat_state"]
# traces['traces_granger'] = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "granger")[0]["same_stat_state"]
schema = plots.trajectories(sin_distribution, traces = traces)

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "paper_sin", "png")

FileNotFoundError: [Errno 2] No such file or directory: 'output_images/paper_sin.png'

In [121]:
#'timepoint_id', 'sample_id'
sin_distribution = pd.read_csv("paper_sin/paper_sin_old_df.csv")
sin_distribution = sin_distribution.rename(
            columns={
                'Unnamed: 0': "timepoint_id",
            }
        )
sin_distribution = sin_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")


traces = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "mean")
traces = traces.rename(
            columns={
                'same_stat_state': "traces_mean",
            }
        )
traces['traces_var'] = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "var")["same_stat_state"]
traces['traces_granger'] = pyciemss.visuals.trajectories.select_traces(sin_distribution, select_by = "granger")["same_stat_state"]
schema = plots.trajectories(sin_distribution, traces = traces)
print(traces_var)
plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")
image = plots.ipy_display(schema, format="PNG").data
save_result(image, "paper_start", "png")


Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8988 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9865, p=1.0000  , df=10
likelihood ratio test: chi2=-792.3666, p=1.0000  , df=10
parameter F test:         F=9832.5499, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8986 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9844, p=1.0000  , df=10
likelihood ratio test: chi2=-779.4979, p=1.0000  , df=10
parameter F test:         F=11157.7647, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.1956 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-81.9749, p=1.0000  , df=10
likelihood ratio test: chi2=-217.5511, p=1.0000  , df=10
parameter F test:         F=5832969.5496, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based F test:         F=-7.8784 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.7535, p=1.0000  , df=10
likelihood ratio test: chi2=-531.0184, p=1.0000  , df=10
parameter F test:         F=180680.8853, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8631 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.5799, p=1.0000  , df=10
likelihood ratio test: chi2=-483.0319, p=1.0000  , df=10
parameter F test:         F=305790.2533, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8977 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9736, p=1.0000  , df=10
likelihood ratio test: chi2=-732.1803, p=1.0000  , df=10
parameter F test:         F=18989.1084, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8779 , p=1.0000  , df_denom=79, df_num=10
ssr 

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based F test:         F=-7.8872 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.8541, p=1.0000  , df=10
likelihood ratio test: chi2=-578.2452, p=1.0000  , df=10
parameter F test:         F=105861.9438, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-3.7004 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-42.1567, p=1.0000  , df=10
likelihood ratio test: chi2=-56.8690, p=1.0000  , df=10
parameter F test:         F=35295813.9609, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8970 , p=1.0000  , df_denom=79, df_num=10
ssr based chi2 test:   chi2=-89.9661, p=1.0000  , df=10
likelihood ratio test: chi2=-709.6945, p=1.0000  , df=10
parameter F test:         F=24424.8915, p=0.0000  , df_denom=79, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.8985 , p=1.0000  , df_denom=79, df_num=10
ssr

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

trajectory  infected_observable_state_var  timepoint_id
timepoint                                              
0                               11.969685             0
1                               15.828713             1
2                               18.445007             2
3                               20.906946             3
4                               23.540363             4
...                                   ...           ...
94                          348138.093750            94
95                          325088.843750            95
96                          302895.062500            96
97                          281833.750000            97
98                          261997.953125            98

[99 rows x 2 columns]

